<h2> TSPEC : multiple orders

<b><i> This is an older notebook that is in need of updating and development

In [ ]:
from pyvista import imred, tv, spectra
import numpy as np
import matplotlib.pyplot as plt
import os

pyvista uses a display tool defined in the tv module. To use the interactive
display in a notebook, set the display to be an external display window, e.g. with 
<code>
%matplotlib qt
</code>
Instantiate a tv object, here we just call it t, but you could call it whatever you want!

In [ ]:
%matplotlib qt
t=tv.TV()

The basic tool for basic image reduction is a Reducer object, defined in the imred module. Instantiate a reducer here. The main argument is an instrument name, which tells it to read a YAML configuration file for the specified instrument. We also give it an optional dir= argument to specify the default directory from which to read images, if a directory is not specified in subsequent commands that read images.

In [ ]:
tspec=imred.Reducer(inst='TSPEC',dir='/home/holtz/red/UT191026/TSPEC',nfowler=8)
a=tspec.reduce(21)
dark=tspec.reduce(22)
t.clear()
t.tv(a)
a=a.subtract(dark)
t.tv(a)

A main method of the reducer object is the reduce() method. Without any additional arguments, reduce() will read an image from disk, subtract the overscan (region(s) as determined from the instrument configuration file), compute an uncertainty array using the gain and readout noise from the instrument configuration file, and return a CCDData object with the data, uncertainty, and mask. 
<p>
To specify the input image, we could pass a string with the file name. If the string does not include a '/', it will read from the default input directory.
<p>
If the file can be identified with a unique integer, then you can just specify this number, which can be very convenient. This is turned into a character string using the formstr attribute define in the configuration file, which is used to search for the file to read.
<p>
We can display the image using the tv() method of our display tool, which can take as input a Data object, and numpy array, or a FITS HDU object.

<h4> Calibration: make and apply flat field

If we add additional arguments to reduce(), we can add additional calibration steps. For example, to flat field the data, we would add a flat= keyword through which we give the reducer a flat field. To add a spatial bias subtraction, we would add a bias= keyword through which we give the reducer a superbias frame.
<br>
First, however, we have to make the calibration products, which is accomplished using the mkflat(), mkbias(), etc methods. These take as input a list of frames to be used to construct the master calibration frame (e.g.. superflat). For ARCES, we will keep the spectral shape of the flats for now.

Create biases and flats. Note that for flats, we have to do scattered light removal, which can be done on reduction of individual images, but since it is slow, we will do it on the combined flat. If we add the display= keyword, giving a display object, then the calibration frames will be displayed, showing each input frame relative to the master frame, so you can inspect and make sure that bad frames are not being included in the combination.

Read and display a star spectral image. For ARCES, we do not apply a flat field here, since we only have flats with the orders, which can move around a bit. Instead, we will use "1D" flats later in the processing.

In [ ]:
crstar=red.crrej(star,crbox='lacosmic',display=t,objlim=50)

<h4> Tracing and extraction

In [ ]:
rows=[[135,235],[295,395],[435,535],[560,660],[735,830]]
apers=[155,316,454,591,761]

t.clear()
t.tv(a)
traces=spectra.Trace(degree=3,rows=rows,lags=range(-75,75),transpose=False)
traces.trace(a,apers,sc0=350,plot=t)


In [ ]:
traces.write('TSPEC_traces.fits')

In [ ]:
traces=spectra.Trace(file='./TSPEC_traces.fits')
vars(traces)

Sum up the arc lamp exposures, get the shift for the existing traces, and extract. Note that if you have a multiprocessor machine, you can specify number of threads to use for the extraction, which will speed things up (but the default threads=0 isn't too terrible).

In [ ]:
import scipy
acr=tspec.reduce(21) #,crbox=[11,1])
t.clear()
t.tv(acr)
traces.pix0=30
order=7
out=traces.extract(acr,rad=20,plot=t)
out.shape
out.data=out.data - scipy.signal.medfilt(out.data,kernel_size=[1,201])

#for aper,row in zip(apers,rows) :
    #out=traces.extract(acr,rad=20,plot=t)
    #out.data=out.data - scipy.signal.medfilt(out.data,kernel_size=[1,201])
    #wcal=spectra.WaveCal(type='chebyshev',orders=[order],degree=3)
    #w=np.atleast_2d(wav[order-3,0,:][::-1])*1.e4
    #bd=np.where(~np.isfinite(w))
    #w[bd[0],bd[1]]=9000.




Do line identification based on previously identified lines, and wavelength fit.

In [ ]:
out.data.shape

In [ ]:
wav=fits.open('/home/holtz/red/tspec_wave.fits')[0].data

for order in range(7,3,-1) : 
    wcal=spectra.WaveCal(type='chebyshev',degree=3,orders=[order])
    wcal.model=wcal.getmod()

    wcal.identify(out[7-order],wav=np.atleast_2d(wav[order-3,0,::-1]*1.e4),
                  thresh=10,file='OHll.dat',plot=True,rad=3)   


In [ ]:
order=7
wcal=spectra.WaveCal(type='chebyshev2D',degree=3,orders=[7,6,5,4,3])
wcal.set_spectrum(out)
wcal.model = wcal.getmod()
vars(wcal)
#wcal=spectra.WaveCal(type='chebyshev',degree=3,orders=[order])


#wav=fits.open('/home/holtz/red/tspec_wave.fits')[0].data
#fig,ax=plots.multi(1,2)
#t.clear() 
#t.tv(wav[::-1,0,::-1])
#t.tv(out)
#wcal.identify(out[7-order],wav=np.atleast_2d(wav[order-3,0,::-1]*1.e4),
#              thresh=10,file='OHll.dat',plot=fig,rad=3)   


wcal.identify(out,wav=np.atleast_2d(wav[::-1,0,::-1]*1.e4),
              thresh=50,file='OHll.dat',plot=True,rad=3)        

In [ ]:
wcal.model


#wcal.wave(image=out[7-order].shape)
#wcal.identify(out[7-order],thresh=50,file='OHll.dat',plot=fig,rad=3)

Now reduce an image

Get shift of traces, and extract. Alternatively, you could use a single call to retrace(), which will do the find() and then trace() using the shifted stored model as a starting guess.

Get the wavelengths for all pixels from the wavelength solution and plot extracted spectra.

In [ ]:
wav.add_wave(imec)
plt.figure()
for row in range(len(imec.wave)) :
    plt.plot(imec.wave[row],imec.data[row])

Resample onto logarithmic wavelength grid and combine orders

In [ ]:
wnew=10**np.arange(3.5,4.0,5.5e-6)
comb=wav.scomb(imec.divide(flat1d),wnew,average=True,usemask=True)
plt.clf()
plt.plot(wnew,comb.data)